<a href="https://colab.research.google.com/github/Kamanga2131/EDSA_projects/blob/main/Titanic_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Titanic Problem

Link for the competition dataset

https://www.kaggle.com/competitions/titanic/data


---

In [1]:
#=======================================================================================
# Data paths
#=======================================================================================
train_path = '/content/train.csv'
test_path = '/content/test.csv'
#=======================================================================================

In [2]:
#=======================================================================================
# Importing the libaries:
#=======================================================================================
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
% matplotlib inline
#=======================================================================================

In [3]:
df = pd.read_csv(train_path)
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print('The columns names are:', '\n', df.columns.values)
print('\n', '_' * 40, '\n')
df.describe()

The columns names are: 
 ['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']

 ________________________________________ 



,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df.info()
print('_'*40 , '\n')
df.describe(include = ['O'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________ 



,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


From the upper info table, we can see that we have a column named PassengerId, This column is as index column for the data, so we can simply drop it as it has no value, and it can be missleading for the algorithm we will use.

And from the lower describtion table, we can conclude that there are no dublicates in the Name feature, and the males on the titanic ship were more than females, and there are 7 dublicate values in ticket feature, and the most frequent Embarked value is 'S'

In [6]:
df.drop('PassengerId', axis = 1, inplace = True)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Yes that looks much better .

We also need to separate the target column from the features.

In [7]:
y = df.Survived
X = df.drop('Survived', axis = 1).copy()
X_train_full, X_val_full, y_train, y_val = train_test_split(X, y, test_size = 0.10, random_state = 2137)
X_train_full.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
119,3,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.2750,NaN,S
554,3,"Ohman, Miss. Velin",female,22.0,0,0,347085,7.7750,NaN,S
375,1,"Meyer, Mrs. Edgar Joseph (Leila Saks)",female,NaN,1,0,PC 17604,82.1708,NaN,C
127,3,"Madsen, Mr. Fridtjof Arne",male,24.0,0,0,C 17369,7.1417,NaN,S
488,3,"Somerton, Mr. Francis William",male,30.0,0,0,A.5. 18509,8.0500,NaN,S


In [8]:
#=======================================================================================
# Setting up the columns
#=======================================================================================
numerical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int64', 'float64']]
low_cardinality_categorical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype == 'object' and X_train_full[col].nunique() < 10]
important_cols = numerical_cols + low_cardinality_categorical_cols
important_cols
#=======================================================================================

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']

In [9]:
X_train = X_train_full[important_cols].copy()
X_val = X_val_full[important_cols].copy()
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
119,3,2.0,4,2,31.2750,female,S
554,3,22.0,0,0,7.7750,female,S
375,1,NaN,1,0,82.1708,female,C
127,3,24.0,0,0,7.1417,male,S
488,3,30.0,0,0,8.0500,male,S


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 119 to 96
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    801 non-null    int64  
 1   Age       642 non-null    float64
 2   SibSp     801 non-null    int64  
 3   Parch     801 non-null    int64  
 4   Fare      801 non-null    float64
 5   Sex       801 non-null    object 
 6   Embarked  799 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 50.1+ KB


In [11]:
X_train.isnull().sum()

Pclass        0
Age         159
SibSp         0
Parch         0
Fare          0
Sex           0
Embarked      2
dtype: int64

We need to:

1.   imputes missing values in numerical data.

2.   imputes missing values and applies a one-hot encoding to categorical data.

so we need to know which features are numerical, and which are categorical




In [12]:
#=======================================================================================
# Setting up transformers
#=======================================================================================
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('OneHotEncoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))
])
#=======================================================================================

In [13]:
#=======================================================================================
# Finishing the preprocessing transformers
#=======================================================================================
preprocessor = ColumnTransformer(
    transformers = [
        ('numerical', numerical_transformer, numerical_cols),
        ('categorical', categorical_transformer, low_cardinality_categorical_cols)
    ])
#=======================================================================================

In [14]:
#=======================================================================================
# Defining the models
#=======================================================================================
def dtc(preprocessor, max_leaf_nodes, X_train, X_val, y_train, y_val):
  dt_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', DecisionTreeClassifier(max_leaf_nodes = max_leaf_nodes))
                              ])
  dt_pipeline.fit(X_train, y_train)
  dt_preds = dt_pipeline.predict(X_val)
  return accuracy_score(y_val, dt_preds)

def rfc(preprocessor, n_estimators, X_train, X_val, y_train, y_val):
  rf_pipleline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', RandomForestClassifier(n_estimators = n_estimators))
                              ])
  rf_pipleline.fit(X_train, y_train)
  rf_preds = rf_pipleline.predict(X_val)
  return accuracy_score(y_val, rf_preds)
  
def svc(preprocessor, C, X_train, X_val, y_train, y_val):
  svc_pipleline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', SVC(C = C))
                              ])
  svc_pipleline.fit(X_train, y_train)
  svc_preds = svc_pipleline.predict(X_val)
  return accuracy_score(y_val, svc_preds)

def gbc(preprocessor, n_estimators, X_train, X_val, y_train, y_val):
  gb_pipleline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', GradientBoostingClassifier(n_estimators = n_estimators))
                              ])
  gb_pipleline.fit(X_train, y_train)
  gb_preds = gb_pipleline.predict(X_val)
  return accuracy_score(y_val, gb_preds)
#=======================================================================================

In [15]:
#=======================================================================================
# Decision Tree Classifier
# Choosing the best max_leaf_nodes, training the model, and calculating the accuracy
#=======================================================================================
dt_ac = {}
for max_leaf_nodes in range(2, 502, 2):
  acc_score = dtc(preprocessor, max_leaf_nodes, X_train, X_val, y_train, y_val)
  dt_ac[max_leaf_nodes] = acc_score

print(f"The maximum accuracy found= {dt_ac[max(dt_ac, key=dt_ac.get)]}, and is corresponding to max_leaf_nodes = {max(dt_ac, key=dt_ac.get)}")
#=======================================================================================

The maximum accuracy found= 0.8222222222222222, and is corresponding to max_leaf_nodes = 80


In [16]:
#=======================================================================================
# Random Forest Classifier
# Choosing the best n_estimators, training the model, and calculating the accuracy
#=======================================================================================
rf_ac = {}
for n_estimators in range(2, 502, 2):
  acc_score = rfc(preprocessor, n_estimators, X_train, X_val, y_train, y_val)
  rf_ac[n_estimators] = acc_score

print(f"The maximum accuracy found= {rf_ac[max(rf_ac, key=rf_ac.get)]}, and is corresponding to n_estimators = {max(rf_ac, key=rf_ac.get)}")
#=======================================================================================

The maximum accuracy found= 0.8, and is corresponding to n_estimators = 2


In [17]:
#=======================================================================================
# Support Vector Classifier
# Choosing the best C, training the model, and calculating the accuracy
#=======================================================================================
svc_ac = {}
for C in [C/10 for C in range(1, 1000, 1)]:
  acc_score = svc(preprocessor, C, X_train, X_val, y_train, y_val)
  svc_ac[C] = acc_score

print(f"The maximum accuracy found= {svc_ac[max(svc_ac, key=svc_ac.get)]}, and is corresponding to C = {max(svc_ac, key=svc_ac.get)}")
#=======================================================================================

The maximum accuracy found= 0.7666666666666667, and is corresponding to C = 88.3


In [18]:
#=======================================================================================
# Gradient Boosting Classifier
# Choosing the best n_estimators, training the model, and calculating the accuracy
#=======================================================================================
gb_ac = {}
for n_estimators in range(2, 502, 2):
  acc_score = gbc(preprocessor, n_estimators, X_train, X_val, y_train, y_val)
  gb_ac[n_estimators] = acc_score

print(f"The maximum accuracy found= {gb_ac[max(gb_ac, key=gb_ac.get)]}, and is corresponding to n_estimators = {max(gb_ac, key=gb_ac.get)}")
#=======================================================================================

The maximum accuracy found= 0.8111111111111111, and is corresponding to n_estimators = 280


## The accuracy = 81.1 % using both Gradient Boosting Classifier and Random Forest Classifier

# <center> Thanks